**1440. Evaluate Boolean Expression**<br>
Write a SQL query to evaluate the boolean expressions in Expressions table.

In [ ]:
SELECT e.left_operand, e.operator, e.right_operand,
CASE WHEN e.operator = '>' AND v1.value>v2.value THEN 'true'
WHEN e.operator = '<' AND v1.value<v2.value THEN 'true'
WHEN e.operator = '=' AND v1.value=v2.value THEN 'true' ELSE 'false' END AS value
FROM Expressions e
JOIN Variables v1
ON e.left_operand = v1.name
JOIN Variables v2
ON e.right_operand = v2.name

**1445. Apples & Oranges**<br>
Write an SQL query to report the difference between number of apples and oranges sold each day.

Return the result table ordered by sale_date in format ('YYYY-MM-DD').

In [ ]:
# Method 1: Self Join

SELECT a.sale_date, b.sold_num-a.sold_num AS diff
FROM Sales a, Sales b
WHERE a.sale_date = b.sale_date
GROUP BY 1
ORDER BY 1

In [ ]:
# Method 2: Case-Function

SELECT sale_date, 
       SUM(CASE WHEN fruit = 'apples' THEN sold_num ELSE -sold_num END) AS diff
FROM sales
GROUP BY sale_date
ORDER BY sale_date

**1454. Active Users**<br>
Write an SQL query to find the id and the name of active users.<br>
Active users are those who logged in to their accounts for 5 or more consecutive days.<br>
Return the result table ordered by the id.

In [ ]:
SELECT t.id, ac.name
FROM Accounts ac
JOIN
(SELECT DISTINCT a.id FROM logins a, logins b 
WHERE a.id=b.id AND DATEDIFF(a.login_date, b.login_date) BETWEEN 1 AND 4
GROUP BY a.id, a.login_date
HAVING COUNT(DISTINCT b.login_date) = 4) t
ON t.id = ac.id
ORDER BY t.id

**1459. Rectangles Area**<br>
Write an SQL query to report of all possible rectangles which can be formed by any two points of the table. 

Each row in the result contains three columns (p1, p2, area) where:

p1 and p2 are the id of two opposite corners of a rectangle and p1 < p2.
Area of this rectangle is represented by the column area.
Report the query in descending order by area in case of tie in ascending order by p1 and p2.

In [ ]:
SELECT a.id AS p1, 
       b.id AS p2, 
       ABS(a.x_value - b.x_value)*ABS(a.y_value - b.y_value) AS area
FROM Points a, Points b
WHERE a.id < b.id
HAVING area != 0
ORDER BY 3 DESC, 1,2

**1468. Calculate Salaries**<br>
Write an SQL query to find the salaries of the employees after applying taxes.

The tax rate is calculated for each company based on the following criteria:
1. 0% If the max salary of any employee in the company is less than 1000 USD.
2. 24% If the max salary of any employee in the company is in the range [1000, 10000] inclusive.
3. 49% If the max salary of any employee in the company is greater than 10000 USD.

Return the result table in any order. Round the salary to the nearest integer.

In [ ]:
WITH taxcalc AS (
SELECT company_id, 
CASE WHEN MAX(salary)>10000 THEN 0.51
WHEN MAX(salary)<1000 THEN 1.00 ELSE 0.76 END AS num
FROM salaries
GROUP BY company_id)

SELECT s.company_id, s.employee_id, s.employee_name, ROUND(s.salary*t.num) AS salary
FROM Salaries s
JOIN taxcalc t
ON s.company_id = t.company_id

**1501. Countries You Can Safely Invest In**<br>
A telecommunications company wants to invest in new countries. The company intends to invest in the countries where the average call duration of the calls in this country is strictly greater than the global average call duration.

Write an SQL query to find the countries where this company can invest.

In [ ]:
SELECT c.name AS country FROM Person p
JOIN Country c
ON c.country_code = SUBSTRING(p.phone_number, 1, 3)
JOIN Calls ca
ON ca.caller_id = p.id OR ca.callee_id = p.id
GROUP BY c.name
HAVING AVG(duration) > (SELECT AVG(duration) FROM calls)

**1532. The Most Recent Three Orders**<br>
Write an SQL query to find the most recent 3 orders of each user. If a user ordered less than 3 orders return all of their orders.

Return the result table sorted by customer_name in ascending order and in case of a tie by the customer_id in ascending order. If there still a tie, order them by the order_date in descending order.

In [ ]:
SELECT t.name AS customer_name, t.customer_id, t.order_id, t.order_date
FROM (SELECT c.name, 
             c.customer_id, 
             o.order_id, 
             o.order_date, 
             ROW_NUMBER() OVER (PARTITION BY o.customer_id ORDER BY o.order_date DESC) AS rown
             FROM Customers c
             JOIN Orders o
             ON c.customer_id = o.customer_id) t
WHERE t.rown<=3
ORDER BY customer_name, customer_id, order_date DESC

**1549. The Most Recent Orders for Each Product**<br>
Write a SQL query to find the most recent order(s) of each product.<br>

Return the result table sorted by product_name in ascending order and in case of a tie by the product_id in ascending order.<br> If there still a tie, order them by the order_id in ascending order.

In [ ]:
# Method 1:

SELECT p.product_name, o.product_id, o.order_id, o.order_date
FROM Orders o 
JOIN Products p
ON o.product_id = p.product_id
WHERE (o.order_date, o.product_id) 
IN (SELECT MAX(order_date) AS order_date, product_id FROM Orders GROUP BY product_id)
ORDER BY p.product_name, o.product_id, o.order_id

In [ ]:
# Method 2:

SELECT t.product_name, t.product_id, t.order_id, t.order_date
FROM (SELECT p.product_name, 
             o.product_id, 
             o.order_id, 
             o.order_date, 
             RANK() OVER (PARTITION BY o.product_id ORDER BY o.order_date DESC) AS rnk
      FROM Products p JOIN Orders o
      ON o.product_id = p.product_id) t
WHERE t.rnk = 1
ORDER BY t.product_name, t.product_id, t.order_id

**1555. Bank Account Summary**<br>
Write an SQL query to report user_id, user_name, credit (current balance after performing transactions),<br>
credit_limit_breached (check credit_limit ("Yes" or "No"))

In [ ]:
SELECT u.user_id, u.user_name, 
       IFNULL((SUM(t.amount) + u.credit),u.credit) AS credit,
       (CASE WHEN IFNULL((SUM(t.amount) + u.credit),u.credit)<0 THEN 'Yes' ELSE 'No' END) AS credit_limit_breached
FROM
    (SELECT paid_by AS user_id, SUM((-1)*amount) AS amount FROM transaction GROUP BY user_id
    UNION
    SELECT paid_to AS user_id, SUM(amount) FROM transaction GROUP BY user_id) t
RIGHT JOIN Users u 
ON u.user_id = t.user_id
GROUP BY u.user_id